In [1]:
!pip config set global.index-url https://mirrors.aliyun.com/pypi/simple/
!python -m pip install -U pip
!pip -V
!pip install sklearn pandas

Writing to /root/.config/pip/pip.conf
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)
pip 20.2.4 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

In [3]:
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.3.1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.2
numpy 1.18.5
pandas 1.1.3
sklearn 0.23.2
tensorflow 2.3.1
tensorflow.keras 2.4.0


In [4]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus))

logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus))

1
1


In [5]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_train_all = x_train_all / 255.0
x_test = x_test / 255.0
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28,1 )
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

In [7]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

batch_size = 128
epochs = 100
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [10]:
model = keras.models.Sequential()
with tf.device(logical_gpus[0].name):
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten())
with tf.device('/CPU:0'):
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalN

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Assert in device /job:localhost/repli

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 7, 7, 128)        

In [ ]:
history = model.fit(train_dataset, 
                    steps_per_epoch = x_train_scaled.shape[0] // batch_size,
                    epochs=10)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/10
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
 37/429 [=>............................] - ETA: 9s - loss: 2.2812 - accuracy: 0.1548Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
 40/429 [=>............................] - ETA: 9s - loss: 2.2794 - accuracy: 0.1617Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
 43/429 [==>...........................] - ETA: 9s - loss: 2.2778 - accuracy: 0.1641Executing op __inference_train_function_1016 in 

 98/429 [=====>........................] - ETA: 7s - loss: 2.1799 - accuracy: 0.2809Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
100/429 [=====>........................] - ETA: 7s - loss: 2.1693 - accuracy: 0.2848Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
103/429 [======>.......................] - ETA: 7s - loss: 2.1543 - accuracy: 0.2889Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in 

Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
163/429 [==========>...................] - ETA: 6s - loss: 1.8405 - accuracy: 0.3863Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
166/429 [==========>...................] - ETA: 6s - loss: 1.8265 - accuracy: 0.3906Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
168/429 [==========>...................] - ETA: 6s - loss: 1.8195 - accuracy: 0.3928Executing op __inference_train_function_1016 in 

223/429 [==============>...............] - ETA: 4s - loss: 1.6156 - accuracy: 0.4556Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
225/429 [==============>...............] - ETA: 4s - loss: 1.6090 - accuracy: 0.4578Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
227/429 [==============>...............] - ETA: 4s - loss: 1.6034 - accuracy: 0.4597Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
230/429 [===============>..............] - ETA: 

...

Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
188/429 [============>.................] - ETA: 5s - loss: 0.4766 - accuracy: 0.8258Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
190/429 [============>.................] - ETA: 5s - loss: 0.4760 - accuracy: 0.8258Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
193/429 [============>.................] - ETA: 5s - loss: 0.4755 - accuracy: 0.8259Executing op __inference_train_function_1016 in 

Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
250/429 [================>.............] - ETA: 4s - loss: 0.4727 - accuracy: 0.8288Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
253/429 [================>.............] - ETA: 4s - loss: 0.4718 - accuracy: 0.8291Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
256/429 [================>.............] - ETA: 4s - loss: 0.4719 - accuracy: 0.8291Executing op __inference_train_function_1016 in 

313/429 [====================>.........] - ETA: 2s - loss: 0.4678 - accuracy: 0.8305Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
316/429 [=====================>........] - ETA: 2s - loss: 0.4675 - accuracy: 0.8305Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
319/429 [=====================>........] - ETA: 2s - loss: 0.4674 - accuracy: 0.8304Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in 

Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
377/429 [=========================>....] - ETA: 1s - loss: 0.4637 - accuracy: 0.8319Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
380/429 [=========================>....] - ETA: 1s - loss: 0.4636 - accuracy: 0.8320Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
383/429 [=========================>....] - ETA: 1s - loss: 0.4637 - accuracy: 0.8318Executing op __inference_train_function_1016 in 

  7/429 [..............................] - ETA: 8s - loss: 0.4433 - accuracy: 0.8426Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
 10/429 [..............................] - ETA: 8s - loss: 0.4516 - accuracy: 0.8383Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
 13/429 [..............................] - ETA: 9s - loss: 0.4457 - accuracy: 0.8383Executing op __inference_train_function_1016 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_1016 in 

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0, 3)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled, y_test)